In [8]:
import pandas as pd
import pymongo

from time import mktime
from dateutil import parser
from time import strptime
from pymongo import MongoClient
from tqdm import tqdm

Make a connection

In [9]:
client = MongoClient(host='192.168.43.225')
db = client.soccerdb

In [10]:
db.matches.find_one()

{'_id': ObjectId('5da395f704f673bbf271d1e3'),
 'competitionId': 102,
 'date': 'July 10, 2016 at 9:00:00 PM GMT+2',
 'dateutc': '2016-07-10 19:00:00',
 'duration': 'ExtraTime',
 'gameweek': 0,
 'groupName': 'Group F',
 'label': 'Portugal - France, 1 - 0 (E)',
 'referees': [{'refereeId': 381617, 'role': 'referee'},
  {'refereeId': 384978, 'role': 'firstAssistant'},
  {'refereeId': 384988, 'role': 'secondAssistant'},
  {'refereeId': 383185, 'role': 'fourthOfficial'}],
 'roundId': 4165474,
 'seasonId': 9291,
 'status': 'Played',
 'teamsData': {'4418': {'coachId': 25549,
   'formation': {'bench': [{'assists': '0',
      'goals': '0',
      'ownGoals': '0',
      'playerId': 7988,
      'redCards': '0',
      'yellowCards': '0'},
     {'assists': '0',
      'goals': '0',
      'ownGoals': '0',
      'playerId': 25552,
      'redCards': '0',
      'yellowCards': '0'},
     {'assists': '0',
      'goals': '0',
      'ownGoals': '0',
      'playerId': 3276,
      'redCards': '0',
      'yellowC

### Add timestamp field to matches documents

In [11]:
for doc in db.matches.find({}, {'wyId': 1, 'date': 1}):
    match_id = doc['wyId']
    date = doc['date']
    date = int(mktime(parser.parse(date).timetuple()))
    db.matches.update_one({'wyId': match_id}, {'$set': {'timestamp': date}})

find every possible mach id and team id 

In [12]:
match2team = db.results.find({}, {'_id': 0, 'matchId': 1, 'teamId': 1})
match2team = [(x['matchId'], x['teamId']) for x in match2team]

find the mean scores before specific match for every team

In [14]:
for match, team in match2team:
    game_time = db.matches.find_one({'wyId': match}, {'_id': 0, 'timestamp': 1})['timestamp']
    previous_scores_json = list(db.matches.find({
        'teamsData.{}'.format(str(team)): {'$exists': True},
        'timestamp': {'$lt': game_time}
    },
    {'_id': 0, 'teamsData.{}.score'.format(str(team)): 1, 
     'teamsData.{}.scoreET'.format(str(team)): 1,
     'teamsData.{}.scoreHT'.format(str(team)): 1,
     'teamsData.{}.scoreP'.format(str(team)): 1}))
    
    scores = [x['teamsData'][str(team)]['score'] for x in previous_scores_json]
    scoresET = [x['teamsData'][str(team)]['scoreET'] for x in previous_scores_json]
    scoresHT = [x['teamsData'][str(team)]['scoreHT'] for x in previous_scores_json]
    scoresP = [x['teamsData'][str(team)]['scoreP'] for x in previous_scores_json]
    
    if scores:
        mean_score = sum(scores) / len(scores)
    else:
        mean_score = 0
    if scoresET:
        mean_scoreET = sum(scoresET) / len(scoresET)
    else:
        mean_scoreET = 0
    if scoresHT:
        mean_scoreHT = sum(scoresHT) / len(scoresHT)
    else:
        mean_scoreHT = 0
    if scoresP:
        mean_scoreP = sum(scoresP) / len(scoresP)
    else:
        mean_scoreP = 0
    
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScore': mean_score}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreET': mean_scoreET}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreHT': mean_scoreHT}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPrevScoreP': mean_scoreP}})
    

### Add overall rating to players dataset

In [15]:
db.players.find_one()

{'_id': ObjectId('5da33c4204f673bbf2661cdd'),
 'birthArea': {'alpha2code': 'TR',
  'alpha3code': 'TUR',
  'id': '792',
  'name': 'Turkey'},
 'birthDate': '1989-06-17',
 'currentNationalTeamId': 4687,
 'currentTeamId': 4502,
 'firstName': 'Harun',
 'foot': 'right',
 'height': 187,
 'lastName': 'Tekin',
 'middleName': '',
 'passportArea': {'alpha2code': 'TR',
  'alpha3code': 'TUR',
  'id': '792',
  'name': 'Turkey'},
 'role': {'code2': 'GK', 'code3': 'GKP', 'name': 'Goalkeeper'},
 'shortName': 'H. Tekin',
 'weight': 78,
 'wyId': 32777}

In [16]:
data_players = pd.read_csv("../data/fifa.csv")
data_players.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [ ]:
for i in tqdm(range(data_players.shape[0])):
    db.players.update_one({'shortName': data_players['Name'].iloc[i]}, {'$set': {'overall': int(data_players['Overall'].iloc[i])}})
    db.players.update_one({'shortName': data_players['Name'].iloc[i]}, {'$set': {'potential': int(data_players['Potential'].iloc[i])}})

 53%|█████▎    | 9586/18207 [04:57<04:07, 34.84it/s]   

In [24]:
db.players.find_one()

{'_id': ObjectId('5d9eeee1cbbd6843f6242c43'),
 'birthArea': {'alpha2code': 'TR',
  'alpha3code': 'TUR',
  'id': '792',
  'name': 'Turkey'},
 'birthDate': '1989-06-17',
 'currentNationalTeamId': 4687,
 'currentTeamId': 4502,
 'firstName': 'Harun',
 'foot': 'right',
 'height': 187,
 'lastName': 'Tekin',
 'middleName': '',
 'overall': 76,
 'passportArea': {'alpha2code': 'TR',
  'alpha3code': 'TUR',
  'id': '792',
  'name': 'Turkey'},
 'potential': 77,
 'role': {'code2': 'GK', 'code3': 'GKP', 'name': 'Goalkeeper'},
 'shortName': 'H. Tekin',
 'weight': 78,
 'wyId': 32777}

### Add average quality of the players for each match and each team

find every possible match id and team id

In [17]:
match2team = db.results.find({}, {'_id': 0, 'matchId': 1, 'teamId': 1})
match2team = [(x['matchId'], x['teamId']) for x in match2team]

Find the mean of players quality of each team

In [66]:
for match, team in tqdm(match2team):
    all_overalls = []
    all_potentials = []
    players = list(db.matches.find({
        'teamsData.{}'.format(str(team)): {'$exists': True}
    },{'_id': 0, 'teamsData.{}.formation.bench.playerId'.format(str(team)): 1, 
       'teamsData.{}.formation.lineup.playerId'.format(str(team)): 1 }))
    playerIdBench = [y['playerId'] for x in players for y in x['teamsData'][str(team)]['formation']['bench']]
    playerIdLineup = [y['playerId'] for x in players for y in x['teamsData'][str(team)]['formation']['lineup']]
    playerId = list(set(playerIdBench + playerIdLineup))
    for player in playerId:
        all_overalls.append(list(db.players.find(
            {'wyId' : player},
            {'_id': 0, 'overall': 1}))[0].get('overall', 0))
        all_potentials.append(list(db.players.find(
            {'wyId' : player},
            {'_id': 0, 'potential': 1}))[0].get('potential', 0))
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPlayerOverall': sum(all_overalls)/len(all_overalls)}})
    db.results.update_one({'matchId': match, 'teamId': team}, {'$set': {'meanPlayerPotential': sum(all_potentials)/len(all_potentials)}})


100%|██████████| 204/204 [02:03<00:00,  1.65it/s]


In [5]:
db.results.find_one()